In [2]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-tw4_4icw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-tw4_4icw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=f13553ac809cc624f9215ab10900a910fbe80665c5053ded38b2c7a339763063
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhn14jof/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [75]:
%%cu

#include <cstdlib>
#include <curand.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <random>

__global__ void addKernel(double* c, double* a, double* b, unsigned int size) {
  for (int i = blockIdx.x * blockDim.x + threadIdx.x; i < size; i += blockDim.x * gridDim.x)
    c[i] = a[i] + b[i];
}


int SUM(double* c, int size){
    int s = 0;
    for (int i = 0; i < size; i += 1)
      s += c[i];
    return s;
}

int main(int argc, char* argv[])
{
    int grid = 2048;
    int block = 1024;
    for (int i = grid; i >= 512; i /= 2){
        for (int j = block; j >= 256; j /= 2){
                for(int n = 1000000;n >= 1000; n/=2){
                  int GRID_SIZE = i;
                  double res1=0;
                  double res2=0;
                  int BLOCK_SIZE = j;
                  printf("n = %d\n", n);
                  printf("GridDim = %d\n", GRID_SIZE);
                  printf("BlockDim = %d\n", BLOCK_SIZE);
                  double time1 = 0;
                  double time2 = 0;
                      int n2b = n * sizeof(double);
                      int n2 = n;

                      double* a = (double*)calloc(n2, sizeof(double));
                      double* b = (double*)calloc(n2, sizeof(double));
                      double* c = (double*)calloc(n2, sizeof(double));
                      double* c2 = (double*)calloc(n2, sizeof(double));

                      for (int i = 0; i < n; i++) {
                          a[i] = -5 + rand() % 11;
                          b[i] =  -5 + rand() % 11;
                      }
                      srand(time(0));
		                  double begin = clock();
                      for (int i = 0; i < n; i++) {
                          c2[i] = a[i]+b[i];
                      }
                      double end = (clock() - begin) / CLOCKS_PER_SEC;
                      time2 += end;
                      double* adev = NULL;
                      cudaError_t cuerr = cudaMalloc((void**)&adev, n2b);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot allocate device array for a: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      double* bdev = NULL;
                      cuerr = cudaMalloc((void**)&bdev, n2b);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot allocate device array for b: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      double* cdev = NULL;
                      cuerr = cudaMalloc((void**)&cdev, n2b);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot allocate device array for c: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cudaEvent_t start, stop;
                      float gpuTime = 0.0f;
                      cuerr = cudaEventCreate(&start);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot create CUDA start event: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaEventCreate(&stop);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot create CUDA end event: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaMemcpy(adev, a, n2b, cudaMemcpyHostToDevice);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot copy a array from host to device: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaMemcpy(bdev, b, n2b, cudaMemcpyHostToDevice);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot copy b array from host to device: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaEventRecord(start, 0);
                      if (cuerr != cudaSuccess) {
                          fprintf(stderr, "Cannot record CUDA event: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      addKernel <<< GRID_SIZE, BLOCK_SIZE >>> (cdev, adev, bdev, n);
                      cuerr = cudaGetLastError();
                      if (cuerr != cudaSuccess)
                      {
                          fprintf(stderr, "Cannot launch CUDA kernel: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaDeviceSynchronize();
                      if (cuerr != cudaSuccess)
                      {
                          fprintf(stderr, "Cannot synchronize CUDA kernel: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaEventRecord(stop, 0);
                      if (cuerr != cudaSuccess)
                      {
                          fprintf(stderr, "Cannot copy c array from device to host: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaMemcpy(c, cdev, n2b, cudaMemcpyDeviceToHost);
                      if (cuerr != cudaSuccess)
                      {
                          fprintf(stderr, "Cannot copy c array from device to host: %s\n",
                              cudaGetErrorString(cuerr));
                          return 0;
                      }
                      cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
                      time1+=gpuTime/1000;
                      cudaEventDestroy(start);
                      cudaEventDestroy(stop);
                      res1=SUM(c,n);
                      res2=SUM(c2,n);
                      cudaFree(adev);
                      cudaFree(bdev);
                      cudaFree(cdev);
                      free(a);
                      free(b);
                      free(c);
                      free(c2);
                  std::cout<<"avg time spent executing GPU: "<<time1<<std::endl;
                  std::cout<<"avg time spent executing CPU: "<<time2<<std::endl;
                  std::cout<<"the difference between GPU and CPU: "<<time2/time1<<std::endl;
                  std::cout<<"sum of GPU: "<<res1<<std::endl;
                  std::cout<<"sum of CPU: "<<res2<<std::endl;
    }
        }
    }
    return 0;
}

n = 1000000
GridDim = 2048
BlockDim = 1024
avg time spent executing GPU: 0.000114656
avg time spent executing CPU: 0.005956
the difference between GPU and CPU: 51.9467
sum of GPU: 7496
sum of CPU: 7496
n = 500000
GridDim = 2048
BlockDim = 1024
avg time spent executing GPU: 8.192e-05
avg time spent executing CPU: 0.003118
the difference between GPU and CPU: 38.0615
sum of GPU: 2464
sum of CPU: 2464
n = 250000
GridDim = 2048
BlockDim = 1024
avg time spent executing GPU: 5.856e-05
avg time spent executing CPU: 0.000784
the difference between GPU and CPU: 13.388
sum of GPU: 1566
sum of CPU: 1566
n = 125000
GridDim = 2048
BlockDim = 1024
avg time spent executing GPU: 5.0112e-05
avg time spent executing CPU: 0.000403
the difference between GPU and CPU: 8.04199
sum of GPU: 455
sum of CPU: 455
n = 62500
GridDim = 2048
BlockDim = 1024
avg time spent executing GPU: 4.3936e-05
avg time spent executing CPU: 0.000207
the difference between GPU and CPU: 4.7114
sum of GPU: 1431
sum of CPU: 1431
n = 3